<a href="https://colab.research.google.com/github/Mutasar/Klasifikasi-Gambar/blob/main/Template_Submission_Akhir.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Proyek Klasifikasi Gambar: [Input Nama Dataset]
- **Nama:** [Input Nama]
- **Email:** [Input Email]
- **ID Dicoding:** [Input Username]

## Import Semua Packages/Library yang Digunakan

In [44]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from statsmodels.tsa.seasonal import seasonal_decompose
from google.colab import files

import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.preprocessing import MultiLabelBinarizer
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input

import numpy as np
import pandas as pd
import io
import requests

## Data Preparation

### Data Loading

In [45]:
# Memuat dataset
df = pd.read_csv("https://raw.githubusercontent.com/Mutasar/Klasifikasi-Gambar/refs/heads/main/photos.csv", sep='\t', on_bad_lines='skip')
display(df.head())

,photo_id,photo_url,photo_image_url,photo_submitted_at,photo_featured,photo_width,photo_height,photo_aspect_ratio,photo_description,photographer_username,...,photo_location_country,photo_location_city,stats_views,stats_downloads,ai_description,ai_primary_landmark_name,ai_primary_landmark_latitude,ai_primary_landmark_longitude,ai_primary_landmark_confidence,blur_hash
0,oSf8ePoG9NU,https://unsplash.com/photos/oSf8ePoG9NU,https://images.unsplash.com/20/frozen-grass.JPG,2013-12-04 14:58:49,t,4000,2667,1.5,NaN,andrekoch,...,NaN,NaN,4296224,30072,black road in between white and brown grass ac...,NaN,NaN,NaN,NaN,LPBM;~RkM|of~VRkNHof%1RjRkof
1,DlsOa5moK4w,https://unsplash.com/photos/DlsOa5moK4w,https://images.unsplash.com/reserve/dRA4UuMBR2...,2014-07-10 17:19:38,t,5184,3456,1.5,Fresh blueberries,majkmmiklavc,...,NaN,NaN,8731049,26411,selective focus of blueberry plant,NaN,NaN,NaN,NaN,LiJ+M4s=-pj;~qx]RpbXx1ogRkob
2,XBGacbT3vXI,https://unsplash.com/photos/XBGacbT3vXI,https://images.unsplash.com/photo-143633523196...,2015-07-08 05:54:38.486126,t,6000,4000,1.5,NaN,shooshanig,...,NaN,NaN,2224540,19250,photography of calm beach side,NaN,NaN,NaN,NaN,LSL}HBR*a{fP_NWVoej[x^s.j[ay
3,FjikPptEbZg,https://unsplash.com/photos/FjikPptEbZg,https://images.unsplash.com/flagged/photo-1441...,2015-08-31 07:12:27.700568,f,3008,2008,1.5,NaN,websanya,...,France,Saint-Étienne-de-Tinée,45536,421,green trees on mountain under white clouds dur...,NaN,NaN,NaN,NaN,LyF7Cux[M{kB?^t6azbHRjjrt7ae
4,PXdBkNF8rlk,https://unsplash.com/photos/PXdBkNF8rlk,https://images.unsplash.com/photo-144596683821...,2015-10-27 17:30:25.724092,t,3000,2000,1.5,Icebergs of Iceland’s Vatnajokull,andersjilden,...,Iceland,NaN,7394191,43442,icebergs floating on body of water during daytime,NaN,NaN,NaN,NaN,LQG]];?b~qayBtIU9Ft79xj[tRM|


### Cleaning Data

In [46]:
# Mengecek dataset menggunakan isnull().sum()

print("Cek nilai null:\n", df.isnull().sum())

Cek nilai null:
 photo_id                              0
photo_url                             0
photo_image_url                       0
photo_submitted_at                    0
photo_featured                        0
photo_width                           0
photo_height                          0
photo_aspect_ratio                    0
photo_description                 13928
photographer_username                 0
photographer_first_name               0
photographer_last_name             1614
exif_camera_make                   2861
exif_camera_model                  2906
exif_iso                           3268
exif_aperture_value                3663
exif_focal_length                  3552
exif_exposure_time                 3288
photo_location_name               25000
photo_location_latitude           17349
photo_location_longitude          17349
photo_location_country            15760
photo_location_city               18550
stats_views                           0
stats_downloads        

In [48]:
df.dropna()

,photo_id,photo_url,photo_image_url,photo_submitted_at,photo_featured,photo_width,photo_height,photo_aspect_ratio,photo_description,photographer_username,...,photo_location_country,photo_location_city,stats_views,stats_downloads,ai_description,ai_primary_landmark_name,ai_primary_landmark_latitude,ai_primary_landmark_longitude,ai_primary_landmark_confidence,blur_hash


In [49]:
columns_to_drop = ['photo_location_name', 'ai_primary_landmark_name', 'ai_primary_landmark_latitude', 'ai_primary_landmark_longitude', 'ai_primary_landmark_confidence']
# Check if columns exist before dropping
existing_columns_to_drop = [col for col in columns_to_drop if col in df.columns]
df = df.drop(columns=existing_columns_to_drop)
print(df.isnull().sum())

photo_id                        0
photo_url                       0
photo_image_url                 0
photo_submitted_at              0
photo_featured                  0
photo_width                     0
photo_height                    0
photo_aspect_ratio              0
photo_description           13928
photographer_username           0
photographer_first_name         0
photographer_last_name       1614
exif_camera_make             2861
exif_camera_model            2906
exif_iso                     3268
exif_aperture_value          3663
exif_focal_length            3552
exif_exposure_time           3288
photo_location_latitude     17349
photo_location_longitude    17349
photo_location_country      15760
photo_location_city         18550
stats_views                     0
stats_downloads                 0
ai_description                  4
blur_hash                       9
dtype: int64


In [50]:
print(df.dtypes)

photo_id                     object
photo_url                    object
photo_image_url              object
photo_submitted_at           object
photo_featured               object
photo_width                   int64
photo_height                  int64
photo_aspect_ratio          float64
photo_description            object
photographer_username        object
photographer_first_name      object
photographer_last_name       object
exif_camera_make             object
exif_camera_model            object
exif_iso                    float64
exif_aperture_value          object
exif_focal_length            object
exif_exposure_time           object
photo_location_latitude     float64
photo_location_longitude    float64
photo_location_country       object
photo_location_city          object
stats_views                   int64
stats_downloads               int64
ai_description               object
blur_hash                    object
dtype: object


In [51]:
print(df.dtypes)

photo_id                     object
photo_url                    object
photo_image_url              object
photo_submitted_at           object
photo_featured               object
photo_width                   int64
photo_height                  int64
photo_aspect_ratio          float64
photo_description            object
photographer_username        object
photographer_first_name      object
photographer_last_name       object
exif_camera_make             object
exif_camera_model            object
exif_iso                    float64
exif_aperture_value          object
exif_focal_length            object
exif_exposure_time           object
photo_location_latitude     float64
photo_location_longitude    float64
photo_location_country       object
photo_location_city          object
stats_views                   int64
stats_downloads               int64
ai_description               object
blur_hash                    object
dtype: object


In [52]:
df['photo_description'] = df['photo_description'].fillna('Unknown')
df['photographer_last_name'] = df['photographer_last_name'].fillna('Unknown')
print(df.isnull().sum())

photo_id                        0
photo_url                       0
photo_image_url                 0
photo_submitted_at              0
photo_featured                  0
photo_width                     0
photo_height                    0
photo_aspect_ratio              0
photo_description               0
photographer_username           0
photographer_first_name         0
photographer_last_name          0
exif_camera_make             2861
exif_camera_model            2906
exif_iso                     3268
exif_aperture_value          3663
exif_focal_length            3552
exif_exposure_time           3288
photo_location_latitude     17349
photo_location_longitude    17349
photo_location_country      15760
photo_location_city         18550
stats_views                     0
stats_downloads                 0
ai_description                  4
blur_hash                       9
dtype: int64


In [53]:
# Impute numerical exif columns with median
numerical_exif_cols = ['exif_iso'] # Based on previous dtypes output, only 'exif_iso' is numerical among exif
for col in numerical_exif_cols:
    if col in df.columns:
        df[col] = df[col].fillna(df[col].median())

# Impute object exif columns with 'Unknown'
object_exif_cols = ['exif_camera_make', 'exif_camera_model', 'exif_aperture_value', 'exif_focal_length', 'exif_exposure_time']
for col in object_exif_cols:
    if col in df.columns:
        df[col] = df[col].fillna('Unknown')

print(df.isnull().sum())

photo_id                        0
photo_url                       0
photo_image_url                 0
photo_submitted_at              0
photo_featured                  0
photo_width                     0
photo_height                    0
photo_aspect_ratio              0
photo_description               0
photographer_username           0
photographer_first_name         0
photographer_last_name          0
exif_camera_make                0
exif_camera_model               0
exif_iso                        0
exif_aperture_value             0
exif_focal_length               0
exif_exposure_time              0
photo_location_latitude     17349
photo_location_longitude    17349
photo_location_country      15760
photo_location_city         18550
stats_views                     0
stats_downloads                 0
ai_description                  4
blur_hash                       9
dtype: int64


In [54]:
# Impute remaining object type columns with 'Unknown'
object_cols_with_nulls = ['ai_description', 'blur_hash']
for col in object_cols_with_nulls:
    if col in df.columns:
        df[col] = df[col].fillna('Unknown')

print(df.isnull().sum())

photo_id                        0
photo_url                       0
photo_image_url                 0
photo_submitted_at              0
photo_featured                  0
photo_width                     0
photo_height                    0
photo_aspect_ratio              0
photo_description               0
photographer_username           0
photographer_first_name         0
photographer_last_name          0
exif_camera_make                0
exif_camera_model               0
exif_iso                        0
exif_aperture_value             0
exif_focal_length               0
exif_exposure_time              0
photo_location_latitude     17349
photo_location_longitude    17349
photo_location_country      15760
photo_location_city         18550
stats_views                     0
stats_downloads                 0
ai_description                  0
blur_hash                       0
dtype: int64


In [55]:
display(df.head())
df.info()

,photo_id,photo_url,photo_image_url,photo_submitted_at,photo_featured,photo_width,photo_height,photo_aspect_ratio,photo_description,photographer_username,...,exif_focal_length,exif_exposure_time,photo_location_latitude,photo_location_longitude,photo_location_country,photo_location_city,stats_views,stats_downloads,ai_description,blur_hash
0,oSf8ePoG9NU,https://unsplash.com/photos/oSf8ePoG9NU,https://images.unsplash.com/20/frozen-grass.JPG,2013-12-04 14:58:49,t,4000,2667,1.5,Unknown,andrekoch,...,Unknown,Unknown,NaN,NaN,NaN,NaN,4296224,30072,black road in between white and brown grass ac...,LPBM;~RkM|of~VRkNHof%1RjRkof
1,DlsOa5moK4w,https://unsplash.com/photos/DlsOa5moK4w,https://images.unsplash.com/reserve/dRA4UuMBR2...,2014-07-10 17:19:38,t,5184,3456,1.5,Fresh blueberries,majkmmiklavc,...,50.0,1/125,NaN,NaN,NaN,NaN,8731049,26411,selective focus of blueberry plant,LiJ+M4s=-pj;~qx]RpbXx1ogRkob
2,XBGacbT3vXI,https://unsplash.com/photos/XBGacbT3vXI,https://images.unsplash.com/photo-143633523196...,2015-07-08 05:54:38.486126,t,6000,4000,1.5,Unknown,shooshanig,...,23.0,1/320,NaN,NaN,NaN,NaN,2224540,19250,photography of calm beach side,LSL}HBR*a{fP_NWVoej[x^s.j[ay
3,FjikPptEbZg,https://unsplash.com/photos/FjikPptEbZg,https://images.unsplash.com/flagged/photo-1441...,2015-08-31 07:12:27.700568,f,3008,2008,1.5,Unknown,websanya,...,18.0,1/80,44.252603,6.917888,France,Saint-Étienne-de-Tinée,45536,421,green trees on mountain under white clouds dur...,LyF7Cux[M{kB?^t6azbHRjjrt7ae
4,PXdBkNF8rlk,https://unsplash.com/photos/PXdBkNF8rlk,https://images.unsplash.com/photo-144596683821...,2015-10-27 17:30:25.724092,t,3000,2000,1.5,Icebergs of Iceland’s Vatnajokull,andersjilden,...,200.0,1/1000,64.044792,-16.173832,Iceland,NaN,7394191,43442,icebergs floating on body of water during daytime,LQG]];?b~qayBtIU9Ft79xj[tRM|


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 26 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   photo_id                  25000 non-null  object 
 1   photo_url                 25000 non-null  object 
 2   photo_image_url           25000 non-null  object 
 3   photo_submitted_at        25000 non-null  object 
 4   photo_featured            25000 non-null  object 
 5   photo_width               25000 non-null  int64  
 6   photo_height              25000 non-null  int64  
 7   photo_aspect_ratio        25000 non-null  float64
 8   photo_description         25000 non-null  object 
 9   photographer_username     25000 non-null  object 
 10  photographer_first_name   25000 non-null  object 
 11  photographer_last_name    25000 non-null  object 
 12  exif_camera_make          25000 non-null  object 
 13  exif_camera_model         25000 non-null  object 
 14  exif_i

### Data Preprocessing

#### Split Dataset

In [56]:
from sklearn.model_selection import train_test_split

# Assuming 'photo_id' is the unique identifier and there is a target variable (replace 'target_column' with your actual target column)
# If you don't have a target column yet, the split will be purely random.
# If you have a target column and want stratified split, uncomment the 'stratify' parameter.

# First split: 85% for training+validation, 15% for test
df_train_val, df_test = train_test_split(df, test_size=0.15, random_state=42) # stratify=df['target_column'] if applicable

# Second split: Split the training+validation set into 70% training and 15% validation (from original data size)
test_size_proportion = 0.15 / (1 - 0.15) # Calculate the proportion of the train_val set to use for validation
df_train, df_val = train_test_split(df_train_val, test_size=test_size_proportion, random_state=42) # stratify=df_train_val['target_column'] if applicable

print("Training set shape:", df_train.shape)
print("Validation set shape:", df_val.shape)
print("Test set shape:", df_test.shape)

Training set shape: (17499, 26)
Validation set shape: (3751, 26)
Test set shape: (3750, 26)


## Modelling

In [57]:

# Example of a simple Sequential CNN model
simple_sequential_model = Sequential([
    # Input Layer
    Input(shape=(224, 224, 3)),
    # Convolutional Layer 1
    Conv2D(32, (3, 3), activation='relu'),
    # Pooling Layer 1
    MaxPooling2D((2, 2)),

    # Convolutional Layer 2
    Conv2D(64, (3, 3), activation='relu'),
    # Pooling Layer 2
    MaxPooling2D((2, 2)),

    # Convolutional Layer 3
    Conv2D(128, (3, 3), activation='relu'),
    # Pooling Layer 3
    MaxPooling2D((2, 2)),

    # Flatten the output for the dense layers
    Flatten(),

    # Dense Layer 1
    Dense(128, activation='relu'),
    # Dropout for regularization (optional)
    Dropout(0.5),

    # Output Layer
    Dense(num_classes, activation='softmax') # Use the defined num_classes
])

# Print the model summary to see the layers
simple_sequential_model.summary()

print("\nThis is an example of a Sequential CNN built from scratch.")
print("In your main project, we are using a Sequential model on top of a pre-trained MobileNetV2 base in cell 719c697a.")

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_19 (Conv2D)              │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_19 (MaxPooling2D) │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_20 (Conv2D)              │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_20 (MaxPooling2D) │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_21 (Conv2D)              │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_21 (MaxPooling2D) │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_5 (Flatten)             │ (None, 86528)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 128)            │    11,075,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 100000)         │    12,900,000 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 24,068,960 (91.82 MB)

 Trainable params: 24,068,960 (91.82 MB)

 Non-trainable params: 0 (0.00 B)


This is an example of a Sequential CNN built from scratch.
In your main project, we are using a Sequential model on top of a pre-trained MobileNetV2 base in cell 719c697a.


In [59]:

# Define image size
IMG_WIDTH = 224
IMG_HEIGHT = 224

# Function to load image from URL
def load_image_from_url(image_url):
    try:
        response = requests.get(image_url, stream=True)
        response.raise_for_status() # Raise an HTTPError for bad responses (4xx or 5xx)
        img = load_img(io.BytesIO(response.content), target_size=(IMG_HEIGHT, IMG_WIDTH))
        img_array = img_to_array(img)
        return img_array / 255.0 # Normalize pixel values
    except Exception as e:
        print(f"Error loading image from {image_url}: {e}")
        return None # Return None for problematic images

# --- Label Preparation (Needs Refinement) ---
# This is a placeholder. You need to finalize your unique label set
# and create the multi-hot encoded labels based on the 'potential_labels' column.

# Example: Get all unique potential labels (this will be a very large set initially)
all_potential_labels = [label for sublist in df['potential_labels'].tolist() for label in sublist]
unique_labels = sorted(list(set(all_potential_labels)))

# For demonstration, let's consider a smaller set of common labels (replace with your actual labels)
# This step requires careful consideration of which labels are relevant and frequent enough.
# You might need to analyze the frequency of terms in 'potential_labels'.
# For this example, I'll just take the first 100 unique labels as a placeholder.
# In a real project, you'd select meaningful labels.
if len(unique_labels) > 100:
    selected_labels = unique_labels[:100] # Placeholder: Select your actual labels
else:
    selected_labels = unique_labels

mlb = MultiLabelBinarizer(classes=selected_labels)

# Fit and transform the potential labels to create multi-hot encoded labels
# Note: This assumes 'potential_labels' column exists and contains lists of strings
# Handle cases where 'potential_labels' might be empty lists or contain labels not in selected_labels
df['multi_hot_labels'] = df['potential_labels'].apply(lambda x: [label for label in x if label in selected_labels])
multi_hot_encoded_labels = mlb.fit_transform(df['multi_hot_labels'])

# Convert to DataFrame for easier handling if needed, or keep as numpy array
multi_hot_encoded_df = pd.DataFrame(multi_hot_encoded_labels, index=df.index, columns=mlb.classes_)

# --- Creating tf.data Dataset ---

# Example: Create a dataset from the training split
# Assuming df_train, df_val, df_test are already created and have 'photo_image_url' and 'multi_hot_labels' columns
# Make sure the indices of multi_hot_encoded_df align with df_train, df_val, df_test

# Align labels with the split dataframes
train_labels = multi_hot_encoded_df.loc[df_train.index].values
val_labels = multi_hot_encoded_df.loc[df_val.index].values
test_labels = multi_hot_encoded_df.loc[df_test.index].values


# Create TensorFlow Datasets
def create_dataset(dataframe, labels):
    image_urls = dataframe['photo_image_url'].values
    # Create a dataset of image URLs and labels
    dataset = tf.data.Dataset.from_tensor_slices((image_urls, labels))

    # Map function to load and preprocess images
    def load_and_preprocess_image(image_url, label):
        # Convert URL tensor to string
        image_url_str = image_url.numpy().decode('utf-8')
        img_array = load_image_from_url(image_url_str)
        if img_array is not None:
            return tf.constant(img_array, dtype=tf.float32), tf.constant(label, dtype=tf.float32)
        else:
            # Handle cases where image loading failed (e.g., return a placeholder or filter out)
            # For now, returning a zero tensor and the label. You might want to filter these out.
            return tf.zeros((IMG_HEIGHT, IMG_WIDTH, 3), dtype=tf.float32), tf.constant(label, dtype=tf.float32)


    # Use tf.py_function for non-TensorFlow operations like requests.get and load_img
    # Ensure the output shapes and dtypes are specified
    dataset = dataset.map(
        lambda x, y: tf.py_function(
            load_and_preprocess_image,
            [x, y],
            [tf.float32, tf.float32]
        ),
        num_parallel_calls=tf.data.AUTOTUNE
    )

    # Filter out problematic images if load_and_preprocess_image returns a specific indicator (e.g., a zero label)
    # Or handle None return from load_image_from_url within the map function

    return dataset

# Create datasets for training, validation, and testing
train_dataset = create_dataset(df_train, train_labels)
val_dataset = create_dataset(df_val, val_labels)
test_dataset = create_dataset(df_test, test_labels)

# Configure datasets for performance
BATCH_SIZE = 32
train_dataset = train_dataset.batch(BATCH_SIZE).prefetch(buffer_size=tf.data.AUTOTUNE)
val_dataset = val_dataset.batch(BATCH_SIZE).prefetch(buffer_size=tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(buffer_size=tf.data.AUTOTUNE)

print("Data pipelines for training, validation, and testing created.")
print(f"Number of training batches: {tf.data.experimental.cardinality(train_dataset).numpy()}")
print(f"Number of validation batches: {tf.data.experimental.cardinality(val_dataset).numpy()}")
print(f"Number of test batches: {tf.data.experimental.cardinality(test_dataset).numpy()}")

# Note: The label preparation part (selecting labels from 'potential_labels') is a placeholder
# and needs to be refined based on your desired classification categories.
# Also, error handling for image loading should be robust.

KeyError: 'potential_labels'

## Evaluasi dan Visualisasi

## Konversi Model

## Inference (Optional)

In [43]:
# Start training the model
# Make sure you have defined the actual number of classes and compiled the model before running this cell.
# Remember to refine the label preparation in the data pipeline (cell 4be73d1f) for your specific multi-label task.

EPOCHS = 10 # You can adjust the number of training epochs

history = model.fit(
    train_dataset,
    epochs=EPOCHS,
    validation_data=val_dataset
)

print("Model training complete.")

ValueError: You must call `compile()` before using the model.